<a href="https://colab.research.google.com/github/ManuelVR461/DuocUCProyectoFinal/blob/master/mercadopublico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from io import BytesIO
import zipfile
import requests

%matplotlib inline

In [ ]:
url = "https://transparenciachc.blob.core.windows.net/lic-da/2023-9.zip"
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    z.extractall()

In [ ]:
df = pd.read_csv("/content/lic_2023-9.csv", encoding="ISO-8859-1", error_bad_lines=False, delimiter=";")
df.shape

<ipython-input-3-a0cccb57d548>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/lic_2023-9.csv", encoding="ISO-8859-1", error_bad_lines=False, delimiter=";")


(1406, 105)

In [ ]:
# Validar la cantidad de duplicados en el DataFrame
cantidad_duplicados = df.duplicated().sum()
# Imprimir la cantidad de duplicados
print("Cantidad de duplicados en el DataFrame:", cantidad_duplicados)

Cantidad de duplicados en el DataFrame: 0


In [ ]:
# Eliminar las filas duplicadas del DataFrame
df = df.drop_duplicates()
df.shape

(1406, 105)

In [ ]:
df.dtypes

Codigo                   int64
Link                    object
CodigoExterno           object
Nombre                  object
Descripcion             object
                         ...  
Valor Total Ofertado    object
CantidadAdjudicada       int64
MontoLineaAdjudica      object
FechaEnvioOferta        object
Oferta seleccionada     object
Length: 105, dtype: object

In [ ]:
col_type_objects = []
col_type_int = []
for column in df:
    if df[column].dtypes == 'object':
        col_type_objects.append(column)
    elif df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
        col_type_int.append(column)
[col_type_objects, col_type_int]

[['Link',
  'CodigoExterno',
  'Nombre',
  'Descripcion',
  'Tipo de Adquisicion',
  'Estado',
  'NombreOrganismo',
  'sector',
  'RutUnidad',
  'NombreUnidad',
  'DireccionUnidad',
  'ComunaUnidad',
  'RegionUnidad',
  'Tipo',
  'CodigoMoneda',
  'Moneda Adquisicion',
  'JustificacionPublicidad',
  'FechaCreacion',
  'FechaCierre',
  'FechaInicio',
  'FechaFinal',
  'FechaPubRespuestas',
  'FechaActoAperturaTecnica',
  'FechaActoAperturaEconomica',
  'FechaPublicacion',
  'FechaAdjudicacion',
  'FechaEstimadaAdjudicacion',
  'FechaSoporteFisico',
  'UnidadTiempoEvaluacion',
  'FechaEstimadaFirma',
  'FechaVisitaTerreno',
  'DireccionVisita',
  'FechaEntregaAntecedentes',
  'DireccionEntrega',
  'FuenteFinanciamiento',
  'MontoEstimado',
  'UnidadTiempo',
  'Modalidad',
  'ProhibicionContratacion',
  'TipoDuracionContrato',
  'JustificacionMontoEstimado',
  'ObservacionContrato',
  'NumeroAprobacion',
  'FechaAprobacion',
  'Rubro1',
  'Rubro2',
  'Rubro3',
  'Nombre producto genrico',

In [ ]:
df.head(5)

,Codigo,Link,CodigoExterno,Nombre,Descripcion,Tipo de Adquisicion,CodigoEstado,Estado,CodigoOrganismo,NombreOrganismo,...,Nombre de la Oferta,Estado Oferta,Cantidad Ofertada,Moneda de la Oferta,MontoUnitarioOferta,Valor Total Ofertado,CantidadAdjudicada,MontoLineaAdjudica,FechaEnvioOferta,Oferta seleccionada
0,9204828,http://www.mercadopublico.cl/fichaLicitacion.h...,94-10-L123,Capacitación enm Fortalecimiento del Trabajo C...,Ejecutar una capacitación en formación en\r\nt...,Licitación Pública Menor a 100 UTM (L1),6,Cerrada,7011,SERVICIO NACIONAL DEL PATRIMONIO CULTURAL,...,Oferta Talent UP Capacitación en Fortalecimien...,Aceptada,8,Peso Chileno,6962,55696,0,0,2023-09-06,No Seleccionada
1,9204828,http://www.mercadopublico.cl/fichaLicitacion.h...,94-10-L123,Capacitación enm Fortalecimiento del Trabajo C...,Ejecutar una capacitación en formación en\r\nt...,Licitación Pública Menor a 100 UTM (L1),6,Cerrada,7011,SERVICIO NACIONAL DEL PATRIMONIO CULTURAL,...,Capacitación en Fortalecimiento del Trabajo co...,Aceptada,39,Peso Chileno,117272,4573608,0,0,2023-09-06,No Seleccionada
2,9204828,http://www.mercadopublico.cl/fichaLicitacion.h...,94-10-L123,Capacitación enm Fortalecimiento del Trabajo C...,Ejecutar una capacitación en formación en\r\nt...,Licitación Pública Menor a 100 UTM (L1),6,Cerrada,7011,SERVICIO NACIONAL DEL PATRIMONIO CULTURAL,...,Capacitación enm Fortalecimiento del Trabajo C...,Aceptada,39,Peso Chileno,138205,5389995,0,0,2023-09-06,No Seleccionada
3,9204828,http://www.mercadopublico.cl/fichaLicitacion.h...,94-10-L123,Capacitación enm Fortalecimiento del Trabajo C...,Ejecutar una capacitación en formación en\r\nt...,Licitación Pública Menor a 100 UTM (L1),6,Cerrada,7011,SERVICIO NACIONAL DEL PATRIMONIO CULTURAL,...,Fortalecimiento del Trabajo Colaborativo y Rel...,Aceptada,39,Peso Chileno,116000,4524000,0,0,2023-09-06,No Seleccionada
4,9204828,http://www.mercadopublico.cl/fichaLicitacion.h...,94-10-L123,Capacitación enm Fortalecimiento del Trabajo C...,Ejecutar una capacitación en formación en\r\nt...,Licitación Pública Menor a 100 UTM (L1),6,Cerrada,7011,SERVICIO NACIONAL DEL PATRIMONIO CULTURAL,...,Oferta CyT Consultores,Aceptada,39,Peso Chileno,94900,3701100,0,0,2023-09-06,No Seleccionada


In [ ]:
print(df['Nombre'][0])
print(df['Descripcion'][0])

Capacitación enm Fortalecimiento del Trabajo Colaborativo y Relaciones Interpersonales en los Equipos
Ejecutar una capacitación en formación en
trabajo en equipo para 39 funcionarios del
Departamento de Gestión y Desarrollo de
Personas


In [ ]:
# Posibles campos para las tablas dimensionales
# dim_tipo_de_adquisicion (id_tipo, Tipo de Adquisicion,)
# dim_estados (CodigoEstado, Estado)
# dim_desc_licitaciones (Codigo, Nombre, Descripcion)
# dim_organismo (CodigoOrganismo, NombreOrganismo, RutUnidad, id_sector)
# dim_sector_organismo (id_sector, sector)
# dim_unidad (CodigoUnidad, NombreUnidad)
# dim_comuna (id_comuna, ComunaUnidad)
# dim_region (id_region, RegionUnidad)
# dim_date

#Tabla de Hecho
#Fact_licitaciones (Codigo, id_tipo, CodigoEstado, CodigoOrganismo, id_sector, CodigoUnidad, id_comuna,id_region, Tipo, Etapas, EstadoPublicidadOfertas, Contrato, CantidadReclamos)

In [ ]:
# Eliminamos las variables innecesarias
df.drop(["Link","CodigoExterno",],axis='columns',inplace=True)

In [ ]:
# Guardar el DataFrame en un nuevo archivo CSV
nombre_del_nuevo_archivo = "/content/dev_lic_2023-8.csv"
df.to_csv(nombre_del_nuevo_archivo, index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')